In [1]:
import pandas as pd
#df = pd.read_csv('Anne_DNA_66.csv')
df = pd.read_csv('Mark_DNA_75.csv')
df = df[df['Class'] != 'interphase']

import numpy as np
np.random.seed(0)

In [2]:
#df_x = pd.read_csv('Anne_DNA_66.csv')
df_x = pd.read_csv('Mark_DNA_75.csv')
trimmed_interphase = df_x[df_x['Class'] == 'interphase'].head(150)
df = pd.concat([df, trimmed_interphase],axis=0) # Add interphase but balanced

In [3]:
y = df['Class'].values

In [4]:
df = df.drop('TableNumber', 1)
df = df.drop('ImageNumber', 1)
df = df.drop('ObjectNumber', 1)
df = df.drop('Class', 1)
df = df.drop('Nuclei_AreaShape_EulerNumber', 1)

In [5]:
df_norm = (df - df.mean()) / (df.max() - df.min())
df_norm.values.shape

(2079, 97)

In [6]:
y

array(['halfcircle', 'halfcircle', 'halfcircle', ..., 'interphase',
       'interphase', 'interphase'], dtype=object)

In [7]:
# Convert y into Y
nb_classes = 23
import numpy as np
from keras.utils import np_utils, generic_utils

# Convert labels to numeric
y_unique = np.unique(y)
dic = {}

for i, label in enumerate(y_unique):
    dic[label] = i
print dic

y_numeric = []
for el in y:
    y_numeric += [dic[el]]
    
y_numeric # now a 2000 label vector
Y = np_utils.to_categorical(y_numeric, nb_classes)

print Y.shape

Y_train = Y

{'halfcircle': 7, 'interphase': 10, 'metaphase': 15, 'earlyprophase': 4, 'debris': 3, 'nucleolirim': 19, 'kidney': 11, 'telophase': 22, 'blurry': 2, 'fragmented': 6, 'apoptotic': 1, 'multinucleate': 18, 'latepro_earlyana': 12, 'prophase': 20, 'anaphase': 0, 'indented': 9, 'monopole': 17, 'holey': 8, 'latetelophase': 13, 'lines': 14, 'micronucleus': 16, 'round': 21, 'elongated': 5}
(2079, 23)


In [8]:
X_train = df_norm.values

In [9]:
from sklearn.cross_validation import StratifiedKFold

#print y_numeric
skl = StratifiedKFold(y_numeric, n_folds=10)

y_numeric = np.array(y_numeric)

i = 0
for train,test in skl:
    print len(train), len(test)
    
X_4 = X_train[train]
Y_4 = Y[train]
y_numeric_4 = y_numeric[train]

X_eval = X_train[test]
Y_eval = Y[test]
y_numeric_eval = y_numeric[test]

X_stratified = np.append(X_4,X_eval,axis=0)
Y_stratified = np.append(Y_4,Y_eval,axis=0)

X_stratified.shape
Y_stratified.shape


1862 217
1864 215
1866 213
1869 210
1870 209
1871 208
1874 205
1876 203
1879 200
1880 199


(2079, 23)

In [10]:
## Convolutional Neural Network with 2 convolutions
import keras
from keras.models import Sequential
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU

nb_epoch = 12

Using TensorFlow backend.


In [11]:
model = Sequential()
model.add(Dense(64, input_dim=97))
#model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
#model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
#model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(23))
model.add(Activation('softmax'))

## Training with 5000 iterations and 0.1 holdout resulted in 66.3 % accuracy!

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim=97))
model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.8))

model.add(Dense(512))
model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.8))

model.add(Dense(4096))
model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=-1, momentum=0.9, weights=None))
model.add(Activation('relu'))
model.add(Dropout(0.8))

model.add(Dense(23))
model.add(Activation('softmax'))

## Training with 5000 iterations and 0.1 holdout resulted in 66.3 % accuracy!

In [12]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [ ]:
import keras.backend as K
session = K.tensorflow_backend._get_session()
session.close()

import tensorflow as tf

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
config=tf.ConfigProto(gpu_options=gpu_options)
session = tf.Session(config=config)
K.tensorflow_backend._set_session(session)

In [36]:
tf_cb = keras.callbacks.TensorBoard(log_dir='./logs/lower_learning_rate/', histogram_freq=0)

# Learning rate lowering
def learning_rate(index):
    if (index > 0):
        return 0.01
    else:
        return 0.05
l_cb = keras.callbacks.LearningRateScheduler(learning_rate)

In [37]:
history = model.fit(X_stratified, Y_stratified, nb_epoch=2000,validation_split=0.1, show_accuracy=True, verbose=2, 
                    callbacks = [tf_cb, l_cb])

Train on 1871 samples, validate on 208 samples
Epoch 1/2000
0s - loss: 0.8239 - acc: 0.7151 - val_loss: 1.2220 - val_acc: 0.6587
Epoch 2/2000
0s - loss: 0.8281 - acc: 0.7114 - val_loss: 1.2072 - val_acc: 0.6779
Epoch 3/2000
0s - loss: 0.7996 - acc: 0.7242 - val_loss: 1.2075 - val_acc: 0.6538
Epoch 4/2000
0s - loss: 0.7968 - acc: 0.7178 - val_loss: 1.2093 - val_acc: 0.6538
Epoch 5/2000
0s - loss: 0.8165 - acc: 0.7092 - val_loss: 1.2092 - val_acc: 0.6779
Epoch 6/2000
0s - loss: 0.7924 - acc: 0.7253 - val_loss: 1.2062 - val_acc: 0.6490
Epoch 7/2000
0s - loss: 0.8211 - acc: 0.7146 - val_loss: 1.2054 - val_acc: 0.6538
Epoch 8/2000
0s - loss: 0.8264 - acc: 0.7119 - val_loss: 1.2112 - val_acc: 0.6587
Epoch 9/2000
0s - loss: 0.8082 - acc: 0.7157 - val_loss: 1.2049 - val_acc: 0.6683
Epoch 10/2000
0s - loss: 0.7882 - acc: 0.7231 - val_loss: 1.2050 - val_acc: 0.6635
Epoch 11/2000
0s - loss: 0.8096 - acc: 0.7194 - val_loss: 1.2140 - val_acc: 0.6587
Epoch 12/2000
0s - loss: 0.7978 - acc: 0.7098 - v

In [ ]:
import skflow
from sklearn import datasets, metrics

clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,4096], n_classes=23, steps=5000, batch_size=len(X_4))
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_without_dropout/')

In [38]:
from sklearn import datasets, metrics
#results = model.predict_classes(X_eval)
#results[0]
score = metrics.accuracy_score(y_numeric_eval, model.predict_classes(X_eval))
print("Accuracy: %f" % score)

199/199 [==============================] - 0s     
Accuracy: 0.668342


In [ ]:
from sklearn import datasets, metrics
iris = datasets.load_iris()
iris.target.shape

In [ ]:
# Creates a graph.
import tensorflow as tf
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,8096], n_classes=23, steps=5000, batch_size=len(X_4), 
                                     learning_rate=0.05, early_stopping_rounds=1000)
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_with_dropout_and_early_stopping/')

In [ ]:
score = metrics.accuracy_score(y_numeric_eval, clf.predict(X_eval))
print("Accuracy: %f" % score)

In [ ]:
clf.save('dropout_DNN_model')

In [18]:
from sklearn.lda import LDA

/home/carmack/.local/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [19]:
lda = LDA()

In [20]:
lda.fit(X_4,y_numeric_4)

/home/carmack/.local/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:453: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/home/carmack/.local/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [21]:
from sklearn import datasets, metrics
score = metrics.accuracy_score(y_numeric_eval, lda.predict(X_eval))
print("Accuracy: %f" % score)

Accuracy: 0.582915


In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_4,y_numeric_4)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
from sklearn import datasets, metrics
score = metrics.accuracy_score(y_numeric_eval, rf.predict(X_eval))
print("Accuracy: %f" % score)

Accuracy: 0.638191


In [24]:
# Real data
# Load the full data
df_full = pd.read_csv('../objects.csv')

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train,y_numeric)
# Get all columns of the training data
#training_values = training_df.columns.values

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
# Get the column labels 
training_values = df.columns.values
df_full_trimmed = df_full[training_values]

In [27]:
# Normalize
df_full_norm = (df_full_trimmed - df_full_trimmed.mean()) / (df_full_trimmed.max() - df_full_trimmed.min())
df_full_norm.values.shape

(454793, 97)

In [ ]:
df_full_norm.describe()

In [ ]:
# Too large!
# rf.predict(df_full_norm.values)

In [28]:
matrix = df_full_norm.values

In [ ]:
matrix.shape

In [ ]:
matrix[0:,:].shape 

In [ ]:
# T

In [29]:
indices = [5000 * i for i in range(92)]

In [33]:
result = np.array([])
for i in range(len(indices) - 1):
    predictions = model.predict_classes(matrix[indices[i]:indices[i+1],:])
    result = np.append(result, predictions)

4793/4793 [==============================] - 0s     


In [34]:
result.shape

(454793,)

In [35]:
np.save('nn_result_24_2.npy',result)

In [ ]:
import numpy as np

In [ ]:
result = np.load('rf_result.npy')

In [ ]:
result

In [ ]:
# Make it categorical to sum up the profiles
from keras.utils import np_utils
Y = np_utils.to_categorical(result, 23)

In [ ]:
import pandas as pd
images = pd.read_csv('../images.csv')

In [ ]:
cc = images[['Image_Metadata_Compound','Image_Metadata_Concentration']]
cc[cc['Image_Metadata_Compound'] == 'AZ-A'][cc['Image_Metadata_Concentration'] == 30.0]

In [ ]:
indexed_images = images.set_index(['TableNumber','ImageNumber'])

In [ ]:
indexed_images

In [ ]:
joined_df = df_full.join(indexed_images, on=['TableNumber','ImageNumber'])

In [ ]:
df_full

In [ ]:
np.unique(df_full['ImageNumber'])

In [ ]:
images.shape

In [16]:
model.save_weights('24-2-11:39.h5')